* 생성자 모델: 가상의 이미지를 판별하는 모델
                
* 판별자 모델: 이미지가 진짜인지 가짜인지 판별하는 모델

* gan 모델: <br>
생성자와 판별자가 결합된 모델<br>
생성자가 생성한 이미지를 판별자에게 판별하도록 하는 모델<br>
생성자가 생성한 이미지를 True 라벨로 학습 => 생성자가 진짜이미지와 같은 이미지를 생성하도록 학습

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input,Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import os
if not os.path.exists('gan_images'):
    os.mkdir('gan_images')

In [6]:
# 생성자

generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation= LeakyReLU(0.2))) # 100차원
generator.add(Reshape((7,7,128))) # 7 7
generator.add(UpSampling2D()) #14 14
generator.add(Conv2D(64, kernel_size=5, padding='same')) #14 14 64
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D()) # 28 28 64
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh')) # 28 28 1
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6272)              633472    
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 14, 14, 128)      0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 64)        204864    
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 14, 14, 64)        0

In [8]:
# 판별자

discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add((Dropout(0.3)))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1,activation='sigmoid'))
discriminator.summary()
discriminator.compile(loss='binary_crossentropy', optimizer='adam')


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        1664      
                                                                 
 activation_1 (Activation)   (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 activation_2 (Activation)   (None, 7, 7, 128)         0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)             

In [10]:
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         840193    
                                                                 
 sequential_2 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1,053,058
Trainable params: 1,052,930
Non-trainable params: 128
_________________________________________________________________


In [ ]:
def gan_train(epoch, batch_size, saving_interval):
    
    (x_train. _), (_,_) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape[0],28,28,1).astype('float32')
    x_train = (x_train-127.5)/ (127.5)
    
    true = np.ones((batch_size,1))
    fake = np.zeros((batch_size,1))
    
    for i in range(epoch):
        idx = np.random.randint(0,x_train.shape[0],batch_size)
        imgs = x_train
        d_loss_real = discriminator.train_on_batch(imgs,true)
        
        noise 